In [30]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from tensorflow.keras.layers import TimeDistributed, Flatten
from tensorflow.keras.layers import LSTM,RepeatVector
from tensorflow.keras.layers import GlobalMaxPooling1D, RepeatVector,Conv1D


# 加载数据
data = pd.read_csv('stock_data.csv',header=None)  # 确保文件路径是正确的
data.sort_index(ascending=False, inplace=True)
data = data.reset_index(drop=True)
data.columns = ['price', 'vol']


# 计算差值
data_diff = data.diff().fillna(0)

# 数据缩放
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_diff)

def generator(data, lookback=200, delay=10, batch_size=32):  # 将delay设置为100
    max_index = len(data) - delay - 1
    i = lookback
    while 1:
        if i + delay >= max_index:
            i = lookback
        samples = np.zeros((batch_size, lookback, data.shape[-1]))
        targets = np.zeros((batch_size, delay, data.shape[-1]))  # Change targets to be 'delay' length sequences
        for j in range(batch_size):
            if i + delay >= max_index:
                i = lookback
            rows = np.arange(i - lookback, i)
            samples[j] = data[rows]
            targets[j] = data[i: i + delay]  # Change targets to be 'delay' length sequences
            i += 1
        yield samples, targets

lookback = 200
delay = 10  # Set delay to 100 for predicting 100 steps ahead
train_gen = generator(data_scaled, lookback=lookback, delay=delay)

model_path = 'stock_model.h5'  # 确保路径是正确的

if os.path.exists(model_path):
    # 加载模型
    model = load_model(model_path)
else:
    # 定义模型
    inputs = Input(shape=(lookback, data.shape[-1]))
    x = MultiHeadAttention(num_heads=2, key_dim=2)(inputs, inputs)
    x = Dropout(0.5)(x)

    # 减小序列长度
    x = Conv1D(64, 10, activation='relu')(x)
    x = GlobalMaxPooling1D()(x)

    # 扩展序列长度到100
    x = RepeatVector(delay)(x)
    x = LSTM(100, return_sequences=True)(x)  # LSTM 层，输出形状为 (batch_size, timesteps, units)

    outputs = TimeDistributed(Dense(data.shape[-1]))(x)  # 用TimeDistributed包装Dense层，以适应新的目标序列长度
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(), loss='mse')
    model.fit(train_gen, epochs=50, steps_per_epoch=20)
    
    # 保存模型
    model.save(model_path)



Epoch 1/50


2023-07-03 16:52:53.455305: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


100/100 [==============================] - 3s 16ms/step - loss: 1.4248
Epoch 2/50
100/100 [==============================] - 2s 16ms/step - loss: 1.0132
Epoch 3/50
100/100 [==============================] - 2s 16ms/step - loss: 0.6269
Epoch 4/50
100/100 [==============================] - 2s 16ms/step - loss: 0.9138
Epoch 5/50
100/100 [==============================] - 2s 16ms/step - loss: 0.7972
Epoch 6/50
100/100 [==============================] - 2s 16ms/step - loss: 1.1415
Epoch 7/50
100/100 [==============================] - 2s 16ms/step - loss: 0.9582
Epoch 8/50
100/100 [==============================] - 2s 16ms/step - loss: 1.2063
Epoch 9/50
100/100 [==============================] - 2s 16ms/step - loss: 0.9067
Epoch 10/50
100/100 [==============================] - 2s 16ms/step - loss: 1.6950
Epoch 11/50
100/100 [==============================] - 2s 16ms/step - loss: 0.6078
Epoch 12/50
100/100 [==============================] - 2s 17ms/step - loss: 0.8771
Epoch 13/50
100/100 [===

/Users/zhangwenbo/miniforge3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [31]:

model.summary()
data_to_predict = data_scaled[-lookback:]  # 最后lookback个数据点
datadiff_to_predict = data_diff[-lookback:]
print(datadiff_to_predict)
samples = np.reshape(data_to_predict, (1, lookback, data.shape[-1]))  # 调整samples的shape为(1, lookback, data.shape[-1])
predictions = model.predict(samples)  # 一次性预测出未来100个时间点的数据
predicted_data = predictions[0]  # 获得预测值
# 对最后100个预测结果进行反归一化
predicted_data_diff = scaler.inverse_transform(predicted_data)
print(predicted_data_diff)

# 使用原始数据的最后一个值作为初始值，然后使用预测的差值序列进行累加
last_value = data.values[-1, :]  # 使用原始数据的最后一个值
predicted_data1 = np.zeros_like(predicted_data_diff)
predicted_data1[0, :] = last_value + predicted_data_diff[0, :]  # 对第一步做特殊处理
for i in range(1, predicted_data_diff.shape[0]):
    predicted_data1[i, :] = predicted_data1[i - 1, :] + predicted_data_diff[i, :]

# 保存预测结果到csv文件
predicted_df = pd.DataFrame(predicted_data1, columns=['predicted_price', 'predicted_vol'])
predicted_df.to_csv('predicted_data_transformer.csv', index=False)


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 200, 2)]             0         []                            
                                                                                                  
 multi_head_attention_4 (Mu  (None, 200, 2)               46        ['input_5[0][0]',             
 ltiHeadAttention)                                                   'input_5[0][0]']             
                                                                                                  
 dropout_10 (Dropout)        (None, 200, 2)               0         ['multi_head_attention_4[0][0]
                                                                    ']                            
                                                                                            